In [ ]:
import pandas as pd
import joblib
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, ClassificationPreset
import datetime
import os

# Load model
model = joblib.load('model.joblib')

# Load new (live) data
df_new = pd.read_csv('new_data.csv')
X_new = df_new.drop('target', axis=1)
y_new = df_new['target']
df_new['prediction'] = model.predict(X_new)

# Load reference data
df_reference = pd.read_csv('reference_data.csv')
df_reference['prediction'] = model.predict(df_reference.drop('target', axis=1))

# Generate report
report = Report(metrics=[DataDriftPreset(), ClassificationPreset()])
report.run(reference_data=df_reference, current_data=df_new)

# Save report
os.makedirs("reports", exist_ok=True)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
report_path = f"reports/report_{timestamp}.html"
report.save_html(report_path)

print(f"Monitoring report saved: {report_path}")